# Data Collection

## Web Scraping URLs

In [1]:
%pip install requests beautifulsoup4 fake_useragent pandas openpyxl

  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
  Using cached fake_useragent-1.5.1-py3-none-any.whl (17 kB)
  Using cached pandas-2.2.2-cp311-cp311-macosx_11_0_arm64.whl (11.3 MB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
  Using cached charset_normalizer-3.3.2-cp311-cp311-macosx_11_0_arm64.whl (118 kB)
  Using cached idna-3.7-py3-none-any.whl (66 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl (121 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl (162 kB)
  Using cached soupsieve-2.5-py3-none-any.whl (36 kB)
  Using cached numpy-2.0.1-cp311-cp311-macosx_11_0_arm64.whl (13.3 MB)
  Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the

In [2]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import urllib.parse

def google_search(query, num_results, time_filter = None):
    ua = UserAgent()
    headers = {'User-Agent': ua.random}

    query = urllib.parse.quote_plus(query)

    google_url = f"https://www.google.com/search?q={query}&num={num_results}"

    if time_filter:
        google_url += f"&tbs={time_filter}"

    response = requests.get(google_url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        search_results = []

        for g in soup.find_all('div', class_='g'):
            anchors = g.find_all('a')
            if anchors:
                link = anchors[0]['href']
                search_results.append(link)
                
        return search_results
    else:
        print(f"failed to retrieve search results: status code {response.status_code}")
        return []


In [14]:
def generate_quarters(start_year, end_year):
    quarters = {}
    if end_year == 2024:
        quarters["2024 Q1"] = "cdr:1,cd_min:1/1/2024,cd_max:3/31/2024"
        quarters["2024 Q2"] = "cdr:1,cd_min:4/1/2024,cd_max:6/30/2024"
        end_year -= 1
    for year in range(start_year, end_year + 1):
        quarters[f"{year} Q1"] = f"cdr:1,cd_min:1/1/{year},cd_max:3/31/{year}"
        quarters[f"{year} Q2"] = f"cdr:1,cd_min:4/1/{year},cd_max:6/30/{year}"
        quarters[f"{year} Q3"] = f"cdr:1,cd_min:7/1/{year},cd_max:9/30/{year}"
        quarters[f"{year} Q4"] = f"cdr:1,cd_min:10/1/{year},cd_max:12/31/{year}"
    return quarters

def generate_query(source_list):
    dictionary = {}
    for source in source_list:
        if source in dictionary:
            continue
        else:
            dictionary[source] = f"singapore industrial property market {source}"
    return dictionary

In [17]:
import pandas as pd

query_dictionary = generate_query(["cna", "singapore business review"])

quarter_dictionary = generate_quarters(2020, 2024)

headers = ["URLs", "Source", "Quarter"]
df = pd.DataFrame(columns=headers)

for source, query in query_dictionary.items():
    for quarter, time_filter in quarter_dictionary.items():
        results = google_search(query, num_results=20, time_filter=time_filter)
        temp_df = pd.DataFrame({"URLs": results, "Source": source, "Quarter": quarter})
        df = pd.concat([df, temp_df], ignore_index=True)

print(df)

query_dictionary = generate_query(["straits times", "business times", "edgeprop"])

quarter_dictionary = generate_quarters(2020, 2024)

for source, query in query_dictionary.items():
    for quarter, time_filter in quarter_dictionary.items():
        results = google_search(query, num_results=30, time_filter=time_filter)
        temp_df = pd.DataFrame({"URLs": results, "Source": source, "Quarter": quarter})
        df = pd.concat([df, temp_df], ignore_index=True)

print(df)

df.to_excel("urls.xlsx", index=False)

                                                  URLs  \
0    https://www.channelnewsasia.com/singapore/mark...   
1          https://www.youtube.com/watch?v=JsMbvmY9FWA   
2    https://www.channelnewsasia.com/singapore/joho...   
3          https://www.youtube.com/watch?v=iJ2Xfc0xnZk   
4    https://m.facebook.com/ChannelNewsAsia/posts/8...   
..                                                 ...   
432  https://plbinsights.com/the-rise-of-self-stora...   
433  https://www.propertyguru.com.sg/property-guide...   
434  https://sbr.com.sg/commercial-property/in-focu...   
435  https://realestateasia.com/videos/proptech-kuc...   
436  https://cosysingapore.com/commercial-property-...   

                        Source  Quarter  
0                          cna  2024 Q1  
1                          cna  2024 Q1  
2                          cna  2024 Q1  
3                          cna  2024 Q1  
4                          cna  2024 Q1  
..                         ...      ...  
432  sing

## URLs Cleaning

In [20]:
df = pd.read_excel("urls.xlsx")
duplicates = df["URLs"].duplicated(keep=False)
df = df[~duplicates]

with pd.ExcelWriter("urls.xlsx", engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name="2020 - 2024", index=False)

## Date Scraping URLs

In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_article_date(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        return f"failed to retrieve the webpage: {e}"
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # locate the date in the meta tag with property 'article:published_time'
    date = soup.find('meta', attrs={'property': 'article:published_time'})
    if date:
        return date['content']
    
    # locate the date in the meta tag with name 'cXenseParse:recs:publishtime'
    date = soup.find('meta', attrs={'name': 'cXenseParse:recs:publishtime'})
    if date:
        return date['content']
    
    # locate the date in the meta tag with name 'article:published_time'
    date = soup.find('meta', attrs={'name': 'article:published_time'})
    if date:
        return date['content']
    
    # locate the date 
    time_element = soup.find('time')
    if time_element and 'datetime' in time_element.attrs:
        return time_element['datetime']
    
    # if not found
    return "date not found"

In [35]:
excel_file = 'urls.xlsx'
sheet_name = '2020 - 2024'
df_existing = pd.read_excel(excel_file, sheet_name=sheet_name)

urls = df_existing['URLs'].tolist()

date_data = []
for url in urls:
    date = get_article_date(url)
    print(date)
    date_data.append({'URLs': url, 'Date': date})

df_date = pd.DataFrame(date_data)

df_updated = pd.merge(df_existing, df_date, on='URLs', how='left')

with pd.ExcelWriter(excel_file, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df_updated.to_excel(writer, sheet_name=sheet_name, index=False)

2024-05-10T07:24:00+08:00
2024-04-12T07:37:00+08:00
2024-05-23T08:00:00+08:00
2024-04-26T13:00:00+08:00
2024-06-25T19:40:00+08:00
2024-04-17T13:34:20+08:00
2024-05-26T05:00:00+08:00
2024-06-28T07:25:00+08:00
2024-04-18T07:25:00+08:00
2024-05-28T19:10:00+08:00
2024-06-19T08:39:00+08:00
date not found
2024-04-25T18:30:00+08:00
2024-04-04T05:00:00+08:00
2024-04-26T17:19:24+08:00
2024-04-25T13:29:11+08:00
2024-06-27T08:44:22+08:00
2024-06-03T11:59:01+08:00
2024-05-29T22:26:22+08:00
2024-04-28T17:36:06+08:00
2024-05-08T10:08:52+08:00
2024-06-16T17:07:29+08:00
2024-05-14T05:00:00+0800
2024-02-07T15:37:42+08:00
2024-03-11T18:22:47+08:00
2024-03-20T13:50:57+08:00
2024-02-06T15:51:56+08:00
2024-03-27T17:18:29+08:00
2024-01-26T02:17:16+08:00
2024-01-05T16:40:00+08:00
2024-01-05T13:08:15+08:00
2024-03-07T14:48:03+08:00
2024-01-27T15:00:00+08:00
2024-01-12T14:00:00+08:00
2024-03-05T12:53:00+08:00
2024-01-02T08:53:00+08:00
2024-01-19T17:47:38+08:00
2024-03-20T15:14:00+08:00
2024-01-13T11:25:00+08:0

In [37]:
excel_file = 'urls.xlsx'
date_df = pd.read_excel(excel_file, sheet_name='2020 - 2024')

filtered_df = date_df[date_df['Date'] == "date not found"]
filtered_df

,URLs,Source,Quarter,Date


## Text Scraping

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

excel_file = 'urls.xlsx'
df = pd.read_excel(excel_file, sheet_name='2020 - 2024')

ImportError: Unable to import required dependencies:
numpy: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.

### Channel NewsAsia

In [1]:
df_cna = df.loc[df['Source'] == "cna"]
urls = df_cna['URLs'].tolist()

cna_data = []

for index, url in enumerate(urls):
    print(f"processing URL {index + 1}/{len(urls)}: {url}")
    try:
        response = requests.get(url)
        if response.status_code == 200:
            html = response.text

            soup = BeautifulSoup(html, 'html.parser')

            paragraphs = soup.find_all('p')
            extracted_text = '\n'.join([p.get_text() for p in paragraphs])

            cna_data.append({'URLs': url, 'Text': extracted_text})
            print(extracted_text[:60])
        else:
            print(f"failed to retrieve {url}. status code: {response.status_code}")
            cna_data.append({'URLs': url, 'Text': ''})
    except Exception as e:
        print(f"error fetching {url}: {str(e)}")
        cna_data.append({'URLs': url, 'Text': ''}) 

cna_text = pd.DataFrame(cna_data)

cna_text.to_excel("text_scraping_urls.xlsx", index=False)

df_updated = pd.merge(df, cna_text, on='URLs', how='left')

NameError: name 'df' is not defined

In [44]:
df_updated

,URLs,Source,Quarter,Date,Text
0,https://www.straitstimes.com/business/singapor...,straits times,2024 Q2,2024-05-10T07:24:00+08:00,NaN
1,https://www.straitstimes.com/business/singapor...,straits times,2024 Q2,2024-04-12T07:37:00+08:00,NaN
2,https://www.straitstimes.com/business/singapor...,straits times,2024 Q2,2024-05-23T08:00:00+08:00,NaN
3,https://www.straitstimes.com/business/singapor...,straits times,2024 Q2,2024-04-26T13:00:00+08:00,NaN
4,https://www.straitstimes.com/business/ramp-up-...,straits times,2024 Q2,2024-06-25T19:40:00+08:00,NaN
...,...,...,...,...,...
302,https://www.businesstimes.com.sg/property/new-...,business times,2020 Q1,2020-02-06T21:50:00+08:00,NaN
303,https://www.businesstimes.com.sg/property/coll...,business times,2020 Q1,2020-01-29T09:04:34+08:00,NaN
304,https://sbr.com.sg/manufacturing/news/seppure-...,singapore business review,2020 Q1,2020-01-08T14:24:50+0800,NaN
305,https://sbr.com.sg/economy/news/inflation-rate...,singapore business review,2020 Q1,2020-01-23T17:29:36+0800,NaN
